In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("mathe_dataset.csv", sep=";", encoding="ISO-8859-1")

In [3]:
df['Keywords'] = df['Keywords'].str.split(',')

In [4]:
df.head()

,Student ID,Student Country,Question ID,Type of Answer,Question Level,Topic,Subtopic,Keywords
0,647,Ireland,77,0,Basic,Statistics,Statistics,"[Stem and Leaf diagram, Relative frequency, Sa..."
1,41,Portugal,77,1,Basic,Statistics,Statistics,"[Stem and Leaf diagram, Relative frequency, Sa..."
2,340,Portugal,77,1,Basic,Statistics,Statistics,"[Stem and Leaf diagram, Relative frequency, Sa..."
3,641,Italy,77,0,Basic,Statistics,Statistics,"[Stem and Leaf diagram, Relative frequency, Sa..."
4,669,Portugal,77,1,Basic,Statistics,Statistics,"[Stem and Leaf diagram, Relative frequency, Sa..."


In [5]:
aggregated_df = df.groupby("Student ID").agg(
    Student_Country=("Student Country", "first"),  # Keep first occurrence
    Correct_Answers=("Type of Answer", "sum"),  # Sum of 1s = correct answers
    Total_Answers=("Type of Answer", "count"),  # Count of total attempts
    Average_Score=("Type of Answer", "mean"),  # Mean = correct/total
    Question_Levels=("Question Level", lambda x: x.unique()),
    Topics=("Topic", lambda x: x.unique()),
    Subtopics=("Subtopic", lambda x: x.unique()),
    Keywords=("Keywords", lambda x: list(set([kw for sublist in x for kw in sublist])))  # Flatten & deduplicate
).reset_index()

In [6]:
aggregated_df.head()

,Student ID,Student_Country,Correct_Answers,Total_Answers,Average_Score,Question_Levels,Topics,Subtopics,Keywords
0,26,Portugal,58,77,0.753247,[Basic],"[Linear Algebra, Differential Equations, Proba...","[Linear Systems, Differential Equations, Proba...","[Cross product, Linear differential equation, ..."
1,28,Portugal,35,59,0.593220,[Basic],"[Statistics, Linear Algebra, Complex Numbers, ...","[Statistics, Linear Systems, Complex Numbers, ...","[Direct inspection, Quantile, Cross product, O..."
2,36,Portugal,18,35,0.514286,"[Basic, Advanced]","[Differentiation, Linear Algebra, Set Theory]","[Derivatives, Linear Systems, Linear Transform...","[Span, Linear dependence, Eigenvalue, Square l..."
3,37,Portugal,87,140,0.621429,"[Basic, Advanced]","[Differentiation, Optimization, Differential E...","[Derivatives, Partial Differentiation, Nonline...","[Power of trigonometric functions, Polygons, S..."
4,41,Portugal,34,59,0.576271,"[Basic, Advanced]","[Statistics, Fundamental Mathematics, Differen...","[Statistics, Algebraic expressions, Equations,...","[Quantile, Cross product, Polygons, Linear dif..."


In [7]:
len(aggregated_df)

372

In [23]:
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

multi_label_cols = ['Topics', 'Subtopics', 'Question_Levels']
preproc_df = aggregated_df.copy()

for col in multi_label_cols:
    preproc_df[col] = preproc_df[col].apply(lambda x: x.split(', ') if isinstance(x, str) else x)
    mlb = MultiLabelBinarizer()
    encoded = mlb.fit_transform(preproc_df[col])
    preproc_df = preproc_df.join(pd.DataFrame(encoded, columns=[f"{col}_{cls}".replace(" ", "_").replace(",", "_") for cls in mlb.classes_]))
    preproc_df.drop(columns=[col], inplace=True)

In [24]:
preproc_df.head()

,Student ID,Student_Country,Correct_Answers,Total_Answers,Average_Score,Keywords,Topics_Analytic_Geometry,Topics_Complex_Numbers,Topics_Differential_Equations,Topics_Differentiation,...,Subtopics_Matrices_and_Determinants,Subtopics_Nonlinear_Optimization,Subtopics_Numerical_Methods,Subtopics_Partial_Differentiation,Subtopics_Probability_,Subtopics_Set_Theory,Subtopics_Statistics,Subtopics_Vector_Spaces,Question_Levels_Advanced,Question_Levels_Basic
0,26,Portugal,58,77,0.753247,"[Cross product, Linear differential equation, ...",1,0,1,1,...,0,0,0,1,1,1,0,0,0,1
1,28,Portugal,35,59,0.593220,"[Direct inspection, Quantile, Cross product, O...",1,1,0,0,...,0,0,0,0,0,0,1,0,0,1
2,36,Portugal,18,35,0.514286,"[Span, Linear dependence, Eigenvalue, Square l...",0,0,0,1,...,1,0,0,0,0,1,0,1,1,1
3,37,Portugal,87,140,0.621429,"[Power of trigonometric functions, Polygons, S...",1,1,1,1,...,0,1,1,1,1,0,0,0,1,1
4,41,Portugal,34,59,0.576271,"[Quantile, Cross product, Polygons, Linear dif...",1,1,1,0,...,0,0,0,0,0,0,1,0,1,1


In [25]:
vectorizer = TfidfVectorizer(max_features=100)
preproc_df['Keywords'] = preproc_df['Keywords'].apply(lambda x: ' '.join(x))
keywords_tfidf = vectorizer.fit_transform(preproc_df['Keywords']).toarray()

In [26]:
keywords_df = pd.DataFrame(keywords_tfidf, columns=[f"keyword_{i}" for i in range(keywords_tfidf.shape[1])])
preproc_df = preproc_df.drop(columns=['Keywords']).reset_index(drop=True)
preproc_df = pd.concat([preproc_df, keywords_df], axis=1)

In [27]:
preproc_df = pd.get_dummies(preproc_df, columns=['Student_Country'], drop_first=True)

In [28]:
preproc_df.head()

,Student ID,Correct_Answers,Total_Answers,Average_Score,Topics_Analytic_Geometry,Topics_Complex_Numbers,Topics_Differential_Equations,Topics_Differentiation,Topics_Fundamental_Mathematics,Topics_Graph_Theory,...,keyword_97,keyword_98,keyword_99,Student_Country_Italy,Student_Country_Lithuania,Student_Country_Portugal,Student_Country_Romania,Student_Country_Russian Federation,Student_Country_Slovenia,Student_Country_Spain
0,26,58,77,0.753247,1,0,1,1,0,0,...,0.112708,0.000000,0.000000,False,False,True,False,False,False,False
1,28,35,59,0.593220,1,1,0,0,0,0,...,0.100546,0.090503,0.091080,False,False,True,False,False,False,False
2,36,18,35,0.514286,0,0,0,1,0,0,...,0.101048,0.000000,0.000000,False,False,True,False,False,False,False
3,37,87,140,0.621429,1,1,1,1,1,0,...,0.000000,0.000000,0.069628,False,False,True,False,False,False,False
4,41,34,59,0.576271,1,1,1,0,1,1,...,0.000000,0.099218,0.099850,False,False,True,False,False,False,False


In [30]:
X = preproc_df.drop(["Student ID", "Average_Score"], axis=1)
y = preproc_df["Average_Score"]

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score

models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
}

In [36]:
results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    
    # Predictions
    y_pred = model.predict(X_test)

    # Metrics
    rmse = mean_squared_error(y_test, y_pred, squared=False)  # Root Mean Squared Error
    r2 = r2_score(y_test, y_pred)  # R² Score
    
    results[name] = {"RMSE": rmse, "R² Score": r2}

C:\Users\suj00\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\suj00\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\suj00\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mea

In [37]:
for model, scores in results.items():
    print(f"{model}: RMSE = {scores['RMSE']:.4f}, R² Score = {scores['R² Score']:.4f}")

Linear Regression: RMSE = 0.3836, R² Score = -1.1139
Random Forest: RMSE = 0.0924, R² Score = 0.8773
Gradient Boosting: RMSE = 0.0944, R² Score = 0.8721
XGBoost: RMSE = 0.0685, R² Score = 0.9327
